In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("../../")
sys.path.append("../")

In [ ]:
from os.path import join, basename
from glob import glob
import numpy as np
import cv2
import matplotlib.pyplot as plt
from typing import List, Any

from photometric.photometric_stereo import estimate_alb_nrm
from utils import load_syn_images, show_single_image, show_multiple_images, load_face_images, show_results, plot_surface, plot_grid_of_surfaces
from photometric.estimate_alb_nrm import estimate_alb_nrm
from photometric.check_integrability import check_integrability
from photometric.construct_surface import construct_surface
from photometric.photometric_stereo import photometric_stereo

Part 1.1.

In [ ]:
albedo, normals, height_map, SE = photometric_stereo(
    image_dir='../SphereGray25/', shadow_trick=True, return_cache=True, show=False
)

In [ ]:
%matplotlib widget

In [ ]:
albedo /= albedo.max()

In [ ]:
plot_surface(height_map, set_lim=True, facecolors=np.dstack([albedo.T, albedo.T, albedo.T]))

In [ ]:
image_dir = "../SphereGray5/"

In [ ]:
# obtain many images in a fixed view under different illumination
[image_stack, scriptV] = load_syn_images(image_dir)
[h, w, n] = image_stack.shape

In [ ]:
# compute the surface gradient from the stack of imgs and light source mat
[albedo, normals] = estimate_alb_nrm(image_stack, scriptV)

In [ ]:
images = [image_stack[:, :, i] for i in range(image_stack.shape[-1])]

In [ ]:
show_multiple_images(
    images, (1, 5), grayscale=True, xticks=False, yticks=False, figsize=(20, 8),
    save=True, path="../results/SphereGray5_original.png"
)

In [ ]:
show_single_image(
    albedo, grayscale=True, xticks=False, yticks=False, save=True, path="../results/SphereGray5_albedo.png"
)

In [ ]:
show_multiple_images(
    [normals[:, :, 0], normals[:, :, 1], normals[:, :, 2]], grid=(1, 3), grayscale=False,
    xticks=False, yticks=False, save=True, path="../results/SphereGray5_normals.png"
)

Part 1.2

In [ ]:
image_dir = "../SphereGray25/"

In [ ]:
# obtain many images in a fixed view under different illumination
[image_stack, scriptV] = load_syn_images(image_dir)
[h, w, n] = image_stack.shape

In [ ]:
image_stack.shape

In [ ]:
# compute the surface gradient from the stack of imgs and light source mat
[albedo, normals] = estimate_alb_nrm(image_stack, scriptV)

In [ ]:
images = [image_stack[:, :, i] for i in range(image_stack.shape[-1])]

In [ ]:
show_multiple_images(
    images, (1, 25), grayscale=True, xticks=False, yticks=False, figsize=(20, 8),
    save=True, path="../results/SphereGray25_original.png"
)

In [ ]:
show_single_image(
    albedo, grayscale=True, xticks=False, yticks=False, save=True, path="../results/SphereGray25_albedo.png"
)

In [ ]:
show_multiple_images(
    [normals[:, :, 0], normals[:, :, 1], normals[:, :, 2]], grid=(1, 3), grayscale=False,
    xticks=False, yticks=False, save=True, path="../results/SphereGray25_normals.png"
)

Varying $n$

In [ ]:
num_images = [1, 2, 3, 6, 9, 12, 15, 18, 21, 25]
all_albedos = []
all_normals = []

In [ ]:
for n in num_images:
    # get the images
    image_substack = image_stack[:, :, :n].copy()
    scriptV_sub = scriptV[:n, :].copy()
    
    # compute the surface gradient from the stack of imgs and light source mat
    [albedo, normals] = estimate_alb_nrm(image_substack, scriptV_sub)
    
    # collect the results
    all_albedos.append(albedo)
    all_normals.append(normals)

In [ ]:
len(all_albedos), len(all_normals)

In [ ]:
fig, ax = plt.subplots(1, len(num_images), figsize=(20, 7))

for i, x in enumerate(all_albedos):
    ax[i].imshow(x, cmap="gray")
    ax[i].set_title(f"$n = {num_images[i]}$")
    ax[i].set_xticks([])
    ax[i].set_yticks([])

plt.savefig("../results/SphereGray25_n_albedos.png", bbox_inches="tight")
plt.show()

In [ ]:
show_multiple_images(
    all_albedos, grid=(1, 8), figsize=(20, 8), grayscale=True,
    xticks=False, yticks=False, save=True, path="../results/SphereGray25_albedo.png",
)

In [ ]:
all_normals[0].shape

In [ ]:
all_xnormals = [x[:, :, 0] for x in all_normals]
show_multiple_images(
    all_xnormals, grid=(1, 8), figsize=(20, 8), grayscale=False,
    xticks=False, yticks=False, save=True, path="../results/SphereGray25_xnormals.png"
)

In [ ]:
all_ynormals = [x[:, :, 1] for x in all_normals]
show_multiple_images(
    all_ynormals, grid=(1, 8), figsize=(20, 8), grayscale=False,
    xticks=False, yticks=False, save=True, path="../results/SphereGray25_ynormals.png",
)

In [ ]:
all_znormals = [x[:, :, -1] for x in all_normals]
show_multiple_images(
    all_znormals, grid=(1, 8), figsize=(20, 8), grayscale=False,
    xticks=False, yticks=False, save=True, path="../results/SphereGray25_znormals.png",
)

Part 1.3: Shadow trick

In [ ]:
image_dir = "../SphereGray25/"

In [ ]:
# obtain many images in a fixed view under different illumination
[image_stack, scriptV] = load_syn_images(image_dir)
[h, w, n] = image_stack.shape

In [ ]:
# compute the surface gradient from the stack of imgs and light source mat
[albedo, normals] = estimate_alb_nrm(image_stack, scriptV)

In [ ]:
# compute the surface gradient from the stack of imgs and light source mat
[albedo_nost, normals_nost] = estimate_alb_nrm(image_stack, scriptV, shadow_trick=False)

In [ ]:
show_multiple_images([albedo, albedo_nost], grayscale=True, grid=(1, 2))

In [ ]:
(albedo == albedo_nost).all()

In [ ]:
show_multiple_images(
    [normals[:, :, 0], normals[:, :, 1], normals[:, :, 2]], grid=(1, 3), grayscale=False,
    xticks=False, yticks=False, save=True, path="../results/SphereGray25_albedo_shadow_trick.png"
)

In [ ]:
show_multiple_images(
    [normals_nost[:, :, 0], normals_nost[:, :, 1], normals[:, :, 2]], grid=(1, 3), grayscale=False,
    xticks=False, yticks=False, save=True, path="../results/SphereGray25_albedo_no_shadow_trick.png"
)